In [1]:
# Uncomment to upgrade packages
# !pip3 install pandas --upgrade --user --quiet
# !pip3 install numpy --upgrade --user --quiet
# !pip3 install scipy --upgrade --user --quiet
# !pip3 install statsmodels --upgrade --user --quiet
# !pip3 install scikit-learn --upgrade --user --quiet
%load_ext autoreload

In [18]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
pd.set_option('precision', 3)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [19]:
# Extra imports
from pandas import read_csv
from sklearn.metrics import confusion_matrix, \
                  classification_report, accuracy_score
from pandas.api.types import CategoricalDtype
from pandas.plotting import scatter_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from numpy.random import  normal, binomial
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families.family import Binomial
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

In [20]:
def confusion(true, pred, classes):
    """
    Function for pretty printing confusion matrices
    """
    cm =pd.DataFrame(confusion_matrix(true, pred), 
                     index=classes,
                     columns=classes)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    return cm

## Logistic regression

 Task: classifying spam mail. When there is an interest in minimizing a particular source of errors how do we change the 'cut point' for prediction?

In [21]:
spam = read_csv("spambase.data", delimiter=',', header=None)
file = open('spambase.names', 'r')
spam.columns = [n.strip() for n in file.readlines()]
spam.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,Class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


We do some basic pre-processing

In [22]:
spam.loc[:,'capital_run_length_average':'capital_run_length_total'] =\
        spam.loc[:,'capital_run_length_average':'capital_run_length_total'].\
                    apply(lambda x: np.log10(x+1))
spam = spam[spam.word_freq_george==0]
spam = spam[spam.word_freq_650==0]
spam = spam[spam.word_freq_hp==0]
spam = spam[spam.word_freq_hpl==0]
spam =spam.drop(columns=['word_freq_george','word_freq_650',
                         'word_freq_hp','word_freq_hpl'])
spam['about_money']=spam.word_freq_free+spam.word_freq_business+\
spam.word_freq_credit+spam.word_freq_money
spam=spam.drop(columns=['word_freq_free','word_freq_business',
                        'word_freq_credit','word_freq_money'])
Class = spam.Class   # move the Class column to the last position
spam=spam.drop(columns=['Class'])
spam['Class'] = Class

spam.shape

(2999, 51)

In [23]:
np.random.seed(4321)
train, test = train_test_split(spam, test_size=0.33)

Now, let's fit a GLM in the learning data

In [24]:
spamM1 = GLM(train.Class, add_constant(train.loc[:,:'about_money']), family=Binomial())
resultM1 = spamM1.fit()
resultM1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Class   No. Observations:                 2009
Model:                            GLM   Df Residuals:                     1958
Model Family:                Binomial   Df Model:                           50
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Mon, 30 Dec 2019   Deviance:                          nan
Time:                        12:51:57   Pearson chi2:                 2.24e+04
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         -5.0624      0.464    -10.904      0.000      -5.972      -4.152
word_freq_make                -0.1299      0.293     -0.443      0.658      -0.705       0.445
word_freq_address             -0.1619      0.131     -1.239      0.216      -0.418       0.094
word_freq_all                 -0.0884      0.167     -0.531      0.595      -0.415       0.238
word_freq_3d                 350.6953    8.8e+05      0.000      1.000   -1.73e+06    1.73e+06
word_freq_our                  0.5344      0.120      4.464      0.000       0.300       0.769
word_freq_over                 0.6943      0.353      1.965      0.049       0.002       1.387
word_freq_remove               1.6118      0.329      4.895      0.000       0.966       2.257
word_freq_internet             0.5349      0.148      3.621      0.000       0.245       0.824
word_freq_order               -0.1082      0.304     -0.356      0.722      -0.703       0.487
word_freq_mail                 0.0561      0.090      0.625      0.532      -0.120       0.232
word_freq_receive             -0.5076      0.380     -1.334      0.182      -1.253       0.238
word_freq_will                -0.1398      0.107     -1.308      0.191      -0.349       0.070
word_freq_people              -0.0531      0.334     -0.159      0.874      -0.708       0.602
word_freq_report               0.0120      0.307      0.039      0.969      -0.590       0.614
word_freq_addresses            0.8487      0.802      1.058      0.290      -0.724       2.421
word_freq_email                0.1439      0.147      0.978      0.328      -0.144       0.432
word_freq_you                  0.1415      0.051      2.782      0.005       0.042       0.241
word_freq_your                 0.1381      0.074      1.870      0.061      -0.007       0.283
word_freq_font                 0.1757      0.279      0.630      0.529      -0.371       0.722
word_freq_000                  1.0192      0.468      2.176      0.030       0.101       1.937
word_freq_lab                -12.4533     10.458     -1.191      0.234     -32.950       8.044
word_freq_labs                -1.6974      1.046     -1.623      0.105      -3.748       0.353
word_freq_telnet              -0.2793      1.874     -0.149      0.882      -3.952       3.394
word_freq_857                 -0.5357      3.463     -0.155      0.877      -7.323       6.252
word_freq_data                -0.4946      0.292     -1.696      0.090      -1.066       0.077
word_freq_415                424.0571   4.52e+07   9.38e-06      1.000   -8.86e+07    8.86e+07
word_freq_85                  -1.8880      1.518     -1.244      0.214      -4.863       1.087
word_freq_technology           0.8304      0.547      1.519      0.129      -0.241       1.902
word_freq_1999                 0.2841      

The variables/features that have small weights are probably not very relevant. Even though we can't do stepwise like in R (Statsmodels does not have stepwise variable selection) we can use crossvalidated Recursive Forward Elimination (RFE) with the implementation of logistic regression from scikit learn. RFE does the same thing as stepwise variable selection but uses accuracy to select the best model using cross validation. Here L1 is used as regularization to make 0 a large number of the weigths and the lower the C the more attributes will be discarded.

In [25]:
logreg = LogisticRegression(solver='liblinear',penalty='l1',C=1)
rfe = RFECV(estimator=logreg,cv=10,n_jobs=-1) 
rfe.fit(train.loc[:,:'about_money'],train.Class);

In [26]:
print('Features Selected:',rfe.n_features_)
print('\n Ranking of features')
sel = pd.DataFrame({'features': train.columns[:-1], 'ranking': rfe.ranking_, 
                    'selected':rfe.support_})
sel.sort_values(by='ranking')

Features Selected: 37

 Ranking of features


,features,ranking,selected
24,word_freq_data,1,True
27,word_freq_technology,1,True
28,word_freq_1999,1,True
30,word_freq_pm,1,True
31,word_freq_direct,1,True
32,word_freq_cs,1,True
33,word_freq_meeting,1,True
34,word_freq_original,1,True
35,word_freq_project,1,True
26,word_freq_85,1,True


We get the extimator from the RFE and the list of selected variable to slice the data matrix

In [27]:
resultM1 = rfe.estimator_
sel_features = list(sel.features[sel.selected])
sel_features

['word_freq_address',
 'word_freq_3d',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_receive',
 'word_freq_will',
 'word_freq_addresses',
 'word_freq_email',
 'word_freq_you',
 'word_freq_font',
 'word_freq_000',
 'word_freq_lab',
 'word_freq_labs',
 'word_freq_data',
 'word_freq_85',
 'word_freq_technology',
 'word_freq_1999',
 'word_freq_pm',
 'word_freq_direct',
 'word_freq_cs',
 'word_freq_meeting',
 'word_freq_original',
 'word_freq_project',
 'word_freq_re',
 'word_freq_edu',
 'word_freq_conference',
 'char_freq_;',
 'char_freq_(',
 'char_freq_!',
 'char_freq_$',
 'char_freq_#',
 'capital_run_length_average',
 'capital_run_length_longest',
 'capital_run_length_total',
 'about_money']

In [29]:
#If the probability is at least 0.5, we can predict spam:
def spam_acc(P=0.5):
    # Accuracy in training
    pred = resultM1.predict_proba(train.loc[:,sel_features])[:,1]
    lab_tr = [1 if i>=P else 0 for i in pred]
    df_tr=confusion(train.Class,lab_tr, classes=['nospam','spam'])

    # Accuracy in test
    pred = resultM1.predict_proba(test.loc[:,sel_features])[:,1]
    lab_ts = [1 if i>=P else 0 for i in pred]
    df_ts=confusion(test.Class,lab_ts, classes=['nospam','spam'])
 
    return df_tr, (1-accuracy_score(train.Class,lab_tr))*100,\
           df_ts, (1-accuracy_score(test.Class,lab_ts))*100

In [30]:
c_tr,e_tr,c_ts,e_ts= spam_acc()
c_tr
print(f'Training error: {e_tr}%')
c_ts
print(f'Test error: {e_ts}%')

Predicted,nospam,spam
Actual,,
nospam,760,89
spam,71,1089


Training error: 7.964161274265802%


Predicted,nospam,spam
Actual,,
nospam,381,42
spam,34,533


Test error: 7.676767676767682%


We can try to lower the probability of predicting spam when it is not (at the expense of increasing the converse probability) by:

In [15]:
c_tr,e_tr,c_ts,e_ts= spam_acc(0.7)
c_tr
print(f'Training error: {e_tr}%')
c_ts
print(f'Test error: {e_ts}%')

Predicted,nospam,spam
Actual,,
nospam,803,46
spam,161,999


Training error: 10.303633648581378%


Predicted,nospam,spam
Actual,,
nospam,402,21
spam,81,486


Test error: 10.303030303030303%


So we get a much better spam filter; notice that the filter has a very low probability of 
predicting spam when it is not (which is the delicate case), of about:

In [16]:
c_ts.loc['nospam','spam'] /c_ts.loc['nospam'].sum()*100

4.964539007092199